In [ ]:
!pip install google-cloud-core
!pip install google-cloud-bigtable

In [3]:
import time
from datetime import datetime as dt

from google.cloud import bigtable
from google.cloud.bigtable import column_family
from google.cloud.bigtable import row_filters

In [4]:
INSTANCE_ID = 'bigtable-1'
TABLE_ID = 'orders_{}'.format(time.time())

### Connecting to BigTable

In [ ]:
client = bigtable.Client.from_service_account_json('../../unbosque-service-account.json', admin = True)

In [ ]:
print('Connecting to database instance')
instance = client.instance(INSTANCE_ID)

### Creating a table

In [ ]:
print('Creating the {} table'.format(TABLE_ID))
table = instance.table(TABLE_ID)

if not table.exists():
    table.create()
else:
    print("ERROR: Table {} already exists".format(TABLE_ID))

In [ ]:
print('Creating all family columns')

customer_cfId = 'customer'
customer_cf = table.column_family(customer_cfId)
customer_cf.create()

items_cfId = 'items'
items_cf = table.column_family(items_cfId)
items_cf.create()

delivery_cfId = 'delivery'
delivery_cf = table.column_family(delivery_cfId)
delivery_cf.create()

### Inserting data

In [ ]:
orders = [
    {
        'orderId': '2497',
        'customer': {
            'firstName': 'Fabian',
            'lastName': 'Penia',
            'customerId': '101'
        },
        'items': [
            {
                'productId': '3301',
                'quantity': '5',
                'unitPrice': '12000'
            },
            {
                'productId': '3302',
                'quantity': '1',
                'unitPrice': '18000'
            },
            {
                'productId': '3303',
                'quantity': '2',
                'unitPrice': '25000'
            }
        ],
        'delivery': {
            'addressLine1': 'Fake Street 123',
            'addressLine2': 'House 10',
            'city': 'Springfield',
            'country': 'US'
        }
    }
]

In [ ]:
print('Writing orders to the table')
rows = []
for order in orders:
    
    dt = dt.utcnow()
    
    row_key = 'order#{}'.format(order['orderId']).encode()
    row = table.direct_row(row_key)
    
    row.set_cell(customer_cfId, 'firstName'.encode(), order['customer']['firstName'], timestamp = dt)
    row.set_cell(customer_cfId, 'lastName'.encode(), order['customer']['lastName'], timestamp = dt)
    row.set_cell(customer_cfId, 'customerId'.encode(), order['customer']['customerId'], timestamp = dt)
    
    for item in order['items']:
        row.set_cell(items_cfId, 'productId'.encode(), item['productId'], timestamp = dt)
        row.set_cell(items_cfId, 'quantity'.encode(), item['quantity'], timestamp = dt)
        row.set_cell(items_cfId, 'unitPrice'.encode(), item['unitPrice'], timestamp = dt)
    
    row.set_cell(delivery_cfId, 'addressLine1'.encode(), order['delivery']['addressLine1'], timestamp = dt)
    row.set_cell(delivery_cfId, 'addressLine2'.encode(), order['delivery']['addressLine2'], timestamp = dt)
    row.set_cell(delivery_cfId, 'city'.encode(), order['delivery']['city'], timestamp = dt)
    row.set_cell(delivery_cfId, 'country'.encode(), order['delivery']['country'], timestamp = dt)
    
    rows.append(row)
    
table.mutate_rows(rows)

### Reading data

In [ ]:
print('Getting a single order by row key')
key = 'order#2497'.encode()

row = table.read_row(key)

firstName = row.cells[customer_cfId]['firstName'.encode()][0].value.decode('utf-8')
lastName = row.cells[customer_cfId]['lastName'.encode()][0].value.decode('utf-8')
customerId = row.cells[customer_cfId]['customerId'.encode()][0].value.decode('utf-8')

# TODO: Get and print items

addressLine1 = row.cells[delivery_cfId]['addressLine1'.encode()][0].value.decode('utf-8')
addressLine2 = row.cells[delivery_cfId]['addressLine2'.encode()][0].value.decode('utf-8')
city = row.cells[delivery_cfId]['city'.encode()][0].value.decode('utf-8')
country = row.cells[delivery_cfId]['country'.encode()][0].value.decode('utf-8')

print(firstName + " | " + lastName + " | " + customerId)
print("\n")
print(addressLine1 + " | " + addressLine2 + " | " + city + " | " + country)

In [ ]:
row.cells

### Deleting table

In [ ]:
print('Deleting the {} table.'.format(TABLE_ID))
table.delete()